# PySpark

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d74ab195d530a5c432d6f0e0e4e432dd7540aa199ee86ee4396410a3866ca061
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [4]:
sc = spark.sparkContext

# Задание 1

In [5]:
!wget https://s3.amazonaws.com/tripdata/201902-citibike-tripdata.csv.zip

--2023-06-11 11:32:09--  https://s3.amazonaws.com/tripdata/201902-citibike-tripdata.csv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.131.168, 52.217.121.176, 54.231.193.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.131.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40466165 (39M) [application/zip]
Saving to: ‘201902-citibike-tripdata.csv.zip’

201902-citibike-tri 100%[===================>]  38.59M  80.4MB/s    in 0.5s    

2023-06-11 11:32:10 (80.4 MB/s) - ‘201902-citibike-tripdata.csv.zip’ saved [40466165/40466165]



In [6]:
!unzip 201902-citibike-tripdata.csv.zip

Archive:  201902-citibike-tripdata.csv.zip
  inflating: 201902-citibike-tripdata.csv  
   creating: __MACOSX/
  inflating: __MACOSX/._201902-citibike-tripdata.csv  


In [7]:
import pandas as pd
df = pd.read_csv('201902-citibike-tripdata.csv')
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,219,2019-02-01 00:00:06.2570,2019-02-01 00:03:46.1090,3494.0,E 115 St & Lexington Ave,40.797911,-73.942300,3501.0,E 118 St & Madison Ave,40.801487,-73.944251,33450,Subscriber,1989,1
1,143,2019-02-01 00:00:28.0320,2019-02-01 00:02:51.7460,438.0,St Marks Pl & 1 Ave,40.727791,-73.985649,236.0,St Marks Pl & 2 Ave,40.728419,-73.987140,25626,Subscriber,1990,1
2,296,2019-02-01 00:01:13.9870,2019-02-01 00:06:10.7340,3571.0,Bedford Ave & Bergen St,40.676368,-73.952918,3549.0,Grand Ave & Bergen St,40.678045,-73.962408,35568,Subscriber,1987,1
3,478,2019-02-01 00:01:14.1520,2019-02-01 00:09:12.7870,167.0,E 39 St & 3 Ave,40.748901,-73.976049,477.0,W 41 St & 8 Ave,40.756405,-73.990026,25045,Subscriber,1964,2
4,225,2019-02-01 00:01:49.3410,2019-02-01 00:05:34.4980,3458.0,W 55 St & 6 Ave,40.763094,-73.978350,3443.0,W 52 St & 6 Ave,40.761330,-73.979820,34006,Subscriber,1979,1


In [ ]:
spark_df = spark.createDataFrame(df)
print(spark_df.printSchema())
print(spark_df.show(truncate=False))

In [ ]:
spark_df.groupBy("start station id").count().sort("count", ascending=False).show() #количество начала поездок для каждой станции

+----------------+-----+
|start station id|count|
+----------------+-----+
|           519.0| 8084|
|           435.0| 6303|
|          3255.0| 5969|
|           497.0| 5903|
|           402.0| 5788|
|           285.0| 5713|
|           490.0| 5035|
|           477.0| 5033|
|           293.0| 4836|
|          3660.0| 4809|
|           459.0| 4733|
|           359.0| 4663|
|           523.0| 4644|
|           499.0| 4547|
|           379.0| 4473|
|          3641.0| 4425|
|           492.0| 4402|
|           465.0| 4333|
|           504.0| 4262|
|           505.0| 4257|
+----------------+-----+
only showing top 20 rows



In [ ]:
spark_df.groupBy("end station id").count().sort("count", ascending=False).show() #количество завершения поездок для каждой станции

+--------------+-----+
|end station id|count|
+--------------+-----+
|         519.0| 8193|
|         402.0| 6688|
|         435.0| 6303|
|        3255.0| 6141|
|         497.0| 5964|
|         285.0| 5794|
|         490.0| 5073|
|         477.0| 5009|
|         293.0| 4990|
|        3660.0| 4827|
|         359.0| 4826|
|         459.0| 4729|
|         523.0| 4696|
|        3263.0| 4622|
|         379.0| 4444|
|        3641.0| 4407|
|         492.0| 4385|
|         465.0| 4364|
|         505.0| 4274|
|         499.0| 4236|
+--------------+-----+
only showing top 20 rows



In [ ]:
!pip install geopandas
import geopandas as gpd
df1 = gpd.read_file("NYC Taxi Zones.geojson")
df1.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."


In [ ]:
df1["longitude"] = df1["geometry"].centroid.x
df1["latitude"] = df1["geometry"].centroid.y
df1.head()

<ipython-input-11-9ce287d42044>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df1["longitude"] = df1["geometry"].centroid.x
<ipython-input-11-9ce287d42044>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df1["latitude"] = df1["geometry"].centroid.y


,shape_area,objectid,shape_leng,location_id,zone,borough,geometry,longitude,latitude
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ...",-74.174000,40.691831
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ...",-73.847422,40.864474
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ...",-73.976968,40.723752
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ...",-74.188484,40.552659


In [ ]:
spark_df1 = spark.createDataFrame(df1[['location_id', 'zone', 'borough', 'longitude', 'latitude']])
print(spark_df1.printSchema())
print(spark_df1.show(truncate=False))

root
 |-- location_id: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)

None
+-----------+-----------------------+-------------+------------------+------------------+
|location_id|zone                   |borough      |longitude         |latitude          |
+-----------+-----------------------+-------------+------------------+------------------+
|1          |Newark Airport         |EWR          |-74.17400027276304|40.691831206401325|
|2          |Jamaica Bay            |Queens       |-73.83129854302213|40.61674529165957 |
|3          |Allerton/Pelham Gardens|Bronx        |-73.84742223236718|40.86447368477527 |
|4          |Alphabet City          |Manhattan    |-73.97696825691759|40.72375214158459 |
|5          |Arden Heights          |Staten Island|-74.18848410184935|40.55265928694546 |
|6          |Arrochar/Fort Wadsworth|Staten Island|-74.07177055895134|

In [ ]:
spark_df1.count()

263

In [34]:
import numpy as np
station_names = set(np.concatenate((df['start station name'].unique(), df['end station name'].unique())))

771


In [35]:
station_dict = dict.fromkeys(station_names, '')
print(station_dict)

{'Fulton St & Grand Ave': '', 'W 107 St & Columbus Ave': '', 'Lispenard St & Broadway': '', 'W 17 St & 9 Ave': '', 'W 52 St & 5 Ave': '', 'Lafayette Ave & St James Pl': '', '3 St & 3 Ave': '', '28 St & 38 Ave': '', 'Bedford Ave & Nassau Ave': '', 'W 106 St & Amsterdam Ave': '', '31 St & 23 Ave': '', 'Willoughby Ave & Walworth St': '', 'Carroll St & Bond St': '', 'W 76 St & Columbus Ave': '', 'Spruce St & Nassau St': '', 'E 84 St & Park Ave': '', '1 Pl & Clinton St': '', 'E 89 St & York Ave': '', '35 Ave & 37 St': '', '5 Ave & E 73 St': '', '9 Ave & W 28 St': '', 'Union Ave & N 12 St': '', 'Jay St & York St': '', 'W 26 St & 10 Ave': '', 'Clermont Ave & Park Ave': '', 'W 31 St & 7 Ave': '', nan: '', '3 Ave & E 95 St': '', 'Rogers Ave & Sterling St': '', 'W 100 St & Broadway': '', 'E 66 St & Madison Ave': '', 'Cadman Plaza E & Johnson St': '', 'E 48 St & 5 Ave': '', 'E 91 St & 2 Ave': '', 'Grand Ave & Bergen St': '', 'Newton Rd & 44 St': '', '1 Ave & E 94 St': '', 'E 59 St & Madison Ave':

In [41]:
from pyspark.sql import functions as F
def find_zone(lon1, lat1): 
  df_closest = spark_df1[['zone', 'longitude', 'latitude']]
  row = df_closest.withColumn("difference", F.abs(F.col('longitude') - lon1) + F.abs(F.col('latitude') - lat1)).sort("difference", ascending=False).first()
  return row.zone

for name in station_names: 
  start = spark_df.filter(spark_df["start station name"]==name).first()
  if start: 
    station_dict[name] = find_zone(start[6], start[5])
  else: 
    end = spark_df.filter(spark_df["end station name"]==name).first()
    station_dict[name] = find_zone(end[10], end[9])
print(station_dict)

{'Fulton St & Grand Ave': 'Charleston/Tottenville', 'W 107 St & Columbus Ave': 'Charleston/Tottenville', 'Lispenard St & Broadway': 'Charleston/Tottenville', 'W 17 St & 9 Ave': 'Charleston/Tottenville', 'W 52 St & 5 Ave': 'Charleston/Tottenville', 'Lafayette Ave & St James Pl': 'Charleston/Tottenville', '3 St & 3 Ave': 'Charleston/Tottenville', '28 St & 38 Ave': 'Charleston/Tottenville', 'Bedford Ave & Nassau Ave': 'Charleston/Tottenville', 'W 106 St & Amsterdam Ave': 'Charleston/Tottenville', '31 St & 23 Ave': 'Charleston/Tottenville', 'Willoughby Ave & Walworth St': 'Charleston/Tottenville', 'Carroll St & Bond St': 'Charleston/Tottenville', 'W 76 St & Columbus Ave': 'Charleston/Tottenville', 'Spruce St & Nassau St': 'Charleston/Tottenville', 'E 84 St & Park Ave': 'Charleston/Tottenville', '1 Pl & Clinton St': 'Charleston/Tottenville', 'E 89 St & York Ave': 'Charleston/Tottenville', '35 Ave & 37 St': 'Charleston/Tottenville', '5 Ave & E 73 St': 'Charleston/Tottenville', '9 Ave & W 28 

In [42]:
import json
with open("stations_zones.json", "w") as outfile:
    json.dump(station_dict, outfile)

In [51]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def translate(mapping):
    def translate_(col):
        return mapping.get(col)
    return udf(translate_, StringType())
spark_df = spark_df.withColumn("start zone", translate(station_dict)("start station name"))
spark_df.show(truncate = False)

+------------+------------------------+------------------------+----------------+---------------------------+----------------------+-----------------------+--------------+---------------------------+--------------------+---------------------+------+----------+----------+------+----------------------+
|tripduration|starttime               |stoptime                |start station id|start station name         |start station latitude|start station longitude|end station id|end station name           |end station latitude|end station longitude|bikeid|usertype  |birth year|gender|start zone            |
+------------+------------------------+------------------------+----------------+---------------------------+----------------------+-----------------------+--------------+---------------------------+--------------------+---------------------+------+----------+----------+------+----------------------+
|219         |2019-02-01 00:00:06.2570|2019-02-01 00:03:46.1090|3494.0          |E 115 St & Le

In [55]:
spark_df = spark_df.withColumn("end zone", translate(station_dict)("end station name"))
spark_df.show(truncate = False)

+------------+------------------------+------------------------+----------------+---------------------------+----------------------+-----------------------+--------------+---------------------------+--------------------+---------------------+------+----------+----------+------+----------------------+----------------------+
|tripduration|starttime               |stoptime                |start station id|start station name         |start station latitude|start station longitude|end station id|end station name           |end station latitude|end station longitude|bikeid|usertype  |birth year|gender|start zone            |end zone              |
+------------+------------------------+------------------------+----------------+---------------------------+----------------------+-----------------------+--------------+---------------------------+--------------------+---------------------+------+----------+----------+------+----------------------+----------------------+
|219         |2019-02-01 

In [56]:
spark_df.groupBy("start zone").count().sort("count", ascending=False).show() #количество начала поездок для каждой зоны

+--------------------+------+
|          start zone| count|
+--------------------+------+
|Charleston/Totten...|938036|
|         City Island|  5699|
|                null|     9|
+--------------------+------+



In [57]:
spark_df.groupBy("end zone").count().sort("count", ascending=False).show() #количество конца поездок для каждой зоны

+--------------------+------+
|            end zone| count|
+--------------------+------+
|Charleston/Totten...|937679|
|         City Island|  6053|
|                null|     9|
|        Far Rockaway|     3|
+--------------------+------+



In [ ]:
!pip install plotly-express

In [62]:
df_end_zones = spark_df.groupBy("end zone").count().sort("count", ascending=False).toPandas().dropna()

In [71]:
latitudes = []
longitudes = []
for zones in df_end_zones['end zone']:
  latitudes.append(spark_df1.filter(F.col("zone")==zones).first().latitude) 
  longitudes.append(spark_df1.filter(F.col("zone")==zones).first().longitude) 
df_end_zones['latitude'] = latitudes 
df_end_zones['longitude'] = longitudes 
df_end_zones.head()

,end zone,count,latitude,longitude
0,Charleston/Tottenville,937679,40.525495,-74.233534
1,City Island,6053,40.849086,-73.781987
3,Far Rockaway,3,40.602433,-73.755244


In [72]:
import plotly
data1 = [dict(type='scattergeo', \
              lat = df_end_zones["latitude"], \
              lon = df_end_zones["longitude"], \
              marker = dict(size = 9, autocolorscale=False, colorscale = 'Viridis', color = df_end_zones['count'], colorbar = dict(title='Amount')) \
              )]
layout1 = dict(title='Number of arrivals to zones',
              geo = dict(scope='usa',projection = dict(type ='albers usa'),showland = True,
                    landcolor="rgb(250,250,250)",subunitcolor = "rgb(217,217,217)",
                    countrycolor = "rgb(217,217,217)",countrywidth =0.5, subunitwidth=0.5))
plotly.offline.iplot({
    "data": data1,
    "layout": layout1
})


# Задание 2

In [26]:
import math
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
def Gaversinus(lat1, lng1, lat2, lng2): 
  R = 6371 
  lat = (lat2 - lat1) * (math.pi / 180)
  lng = (lng2 - lng1) * (math.pi / 180)
  rez = math.sin(lat / 2) * math.sin(lat / 2) + math.cos(lat1 * (math.pi / 180)) * math.cos(lat2 * (math.pi / 180)) * math.sin(lng / 2) * math.sin(lng / 2);
  d = 2 * R * math.atan2(math.sqrt(rez), math.sqrt(1-rez));
  return d 

calc_dist_udf = F.udf(Gaversinus, DoubleType())
spark_df_with_dists = spark_df.filter(F.col("start station name") != F.col("end station name")).withColumn('gs_dist', calc_dist_udf(F.col("start station latitude"), F.col("start station longitude"), F.col("end station latitude"), F.col("end station longitude")))
spark_df_with_dists.show()

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+-------------------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|            gs_dist|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+-------------------+
|         219|2019-02-01 00:00:...|2019-02-01 00:03:...|          3494.0|E 115 St & Lexing...|             40.797911|               -73.9423|        3501.0|E 118 St & Ma

In [27]:
spark_df_with_dists.agg(F.min('gs_dist').alias("Минимальное расстояние"), F.max('gs_dist').alias("Максимальное расстояние"), F.avg('gs_dist').alias("Среднее расстояние"), F.median('gs_dist').alias("Медиана"), F.stddev_pop('gs_dist').alias("Стандартное отклонение")).show()

+----------------------+-----------------------+------------------+------------------+----------------------+
|Минимальное расстояние|Максимальное расстояние|Среднее расстояние|           Медиана|Стандартное отклонение|
+----------------------+-----------------------+------------------+------------------+----------------------+
|   0.03673581134308801|     15.326431486694728|1.6403749148456084|1.2628745712443081|     1.288676481830412|
+----------------------+-----------------------+------------------+------------------+----------------------+



# Задание 3

Среднее количество начала поездок в день для каждой из станции

In [66]:
from pyspark.sql.functions import date_format
spark_df.withColumn("startday", date_format("starttime", "yyyy-MM-dd"))\
.groupBy("start station name", "startday")\
.count()\
.groupBy("start station name")\
.avg("count")\
.show()

+--------------------+------------------+
|  start station name|        avg(count)|
+--------------------+------------------+
|       45 Rd & 11 St|21.821428571428573|
|       10 St & 5 Ave|11.535714285714286|
|  N 11 St & Kent Ave|14.821428571428571|
| Berkeley Pl & 6 Ave|17.178571428571427|
|   Dean St & Hoyt St|15.964285714285714|
|    W 12 St & W 4 St|62.392857142857146|
| Park Ave & E 124 St|54.392857142857146|
|      49 Ave & 21 St| 7.678571428571429|
|     E 91 St & 2 Ave|39.392857142857146|
|Park Pl & Vanderb...|33.892857142857146|
|Central Park West...| 74.92857142857143|
|    Jay St & Tech Pl| 36.92857142857143|
| Bank St & Hudson St|62.785714285714285|
|       44 Dr & 21 St|22.321428571428573|
|Meserole Ave & Ma...| 39.92857142857143|
|Sterling St & Bed...| 4.555555555555555|
|  E 91 St & Park Ave|22.892857142857142|
|Carlton Ave & Flu...|17.571428571428573|
|39 St & 2 Ave - C...| 6.285714285714286|
|E 123 St & Lexing...|             11.25|
+--------------------+------------

Среднее количество завершения поездок в день для каждой станции

In [67]:
spark_df.withColumn("endday", date_format("stoptime", "yyyy-MM-dd"))\
.groupBy("end station name", "endday")\
.count()\
.groupBy("end station name")\
.avg("count")\
.show()

+--------------------+------------------+
|    end station name|        avg(count)|
+--------------------+------------------+
|       45 Rd & 11 St|21.178571428571427|
|       10 St & 5 Ave|11.642857142857142|
|  N 11 St & Kent Ave|15.428571428571429|
| Berkeley Pl & 6 Ave|17.178571428571427|
|   Dean St & Hoyt St|16.642857142857142|
|    W 12 St & W 4 St| 65.07142857142857|
| Park Ave & E 124 St| 54.82142857142857|
|      49 Ave & 21 St|               8.0|
|     E 91 St & 2 Ave|40.785714285714285|
|Park Pl & Vanderb...|              29.0|
|Central Park West...|              69.0|
|    Jay St & Tech Pl| 37.96551724137931|
| Bank St & Hudson St|             64.25|
|       44 Dr & 21 St|22.107142857142858|
|Meserole Ave & Ma...|39.607142857142854|
|Sterling St & Bed...| 4.777777777777778|
|  E 91 St & Park Ave|             23.25|
|Carlton Ave & Flu...|22.571428571428573|
|39 St & 2 Ave - C...| 6.925925925925926|
|E 123 St & Lexing...|11.321428571428571|
+--------------------+------------

Среднее количество начала поездок в утренний период для каждой станции (6AM - 12PM)

In [71]:
from pyspark.sql import functions as F
start_morning_spark_df = spark_df.withColumn("starttimet", date_format("starttime", 'HH:mm:ss'))\
.filter(F.col("starttimet") >= "06:00:00")\
.filter(F.col("starttimet") <= "11:59:00")\
.groupBy("start station name")\
.count()\
.groupBy("start station name")\
.avg("count")

start_morning_spark_df.show()

+--------------------+----------+
|  start station name|avg(count)|
+--------------------+----------+
|       45 Rd & 11 St|     131.0|
|       10 St & 5 Ave|     103.0|
|  N 11 St & Kent Ave|      58.0|
| Park Ave & E 124 St|     679.0|
|      49 Ave & 21 St|     120.0|
|    W 12 St & W 4 St|     617.0|
| Berkeley Pl & 6 Ave|     208.0|
|   Dean St & Hoyt St|     117.0|
|     E 91 St & 2 Ave|     481.0|
| Bank St & Hudson St|     729.0|
|    Jay St & Tech Pl|     225.0|
|Central Park West...|     821.0|
|Meserole Ave & Ma...|     371.0|
|Park Pl & Vanderb...|     294.0|
|Sterling St & Bed...|      57.0|
|       44 Dr & 21 St|     105.0|
|Carlton Ave & Flu...|      98.0|
|  E 91 St & Park Ave|     211.0|
|39 St & 2 Ave - C...|      43.0|
|Washington St & G...|     667.0|
+--------------------+----------+
only showing top 20 rows



Среднее количество завершения поездок в утренний период для каждой станции (6AM - 12PM)

In [72]:
from pyspark.sql.functions import date_format
from pyspark.sql import functions as F
end_morning_spark_df = spark_df.withColumn("stoptimet", date_format("stoptime", 'HH:mm:ss'))\
.filter(F.col("stoptimet") >= "06:00:00")\
.filter(F.col("stoptimet") <= "11:59:00")\
.groupBy("end station name")\
.count()\
.groupBy("end station name")\
.avg("count")

end_morning_spark_df.show()

+--------------------+----------+
|    end station name|avg(count)|
+--------------------+----------+
|       10 St & 5 Ave|      86.0|
|       45 Rd & 11 St|     238.0|
|  N 11 St & Kent Ave|     154.0|
| Park Ave & E 124 St|     737.0|
|    W 12 St & W 4 St|     461.0|
|   Dean St & Hoyt St|     119.0|
| Berkeley Pl & 6 Ave|     118.0|
|      49 Ave & 21 St|      52.0|
|     E 91 St & 2 Ave|     217.0|
|       44 Dr & 21 St|     256.0|
| Bank St & Hudson St|     404.0|
|Central Park West...|     373.0|
|Park Pl & Vanderb...|     297.0|
|    Jay St & Tech Pl|     455.0|
|Meserole Ave & Ma...|     281.0|
|Sterling St & Bed...|      18.0|
|Carlton Ave & Flu...|     263.0|
|  E 91 St & Park Ave|     190.0|
|39 St & 2 Ave - C...|      67.0|
|Washington St & G...|     747.0|
+--------------------+----------+
only showing top 20 rows



Среднее количество начала поездок в дневной период для каждой станции (12PM - 6PM)

In [73]:
from pyspark.sql import functions as F
start_day_spark_df = spark_df.withColumn("starttimet", date_format("starttime", 'HH:mm:ss'))\
.filter(F.col("starttimet") >= "12:00:00")\
.filter(F.col("starttimet") <= "17:59:00")\
.groupBy("start station name")\
.count()\
.groupBy("start station name")\
.avg("count")

start_day_spark_df.show()

+--------------------+----------+
|  start station name|avg(count)|
+--------------------+----------+
|       10 St & 5 Ave|     141.0|
|       45 Rd & 11 St|     235.0|
|  N 11 St & Kent Ave|     181.0|
|    W 12 St & W 4 St|     688.0|
| Park Ave & E 124 St|     381.0|
| Berkeley Pl & 6 Ave|     153.0|
|      49 Ave & 21 St|      62.0|
|   Dean St & Hoyt St|     203.0|
|     E 91 St & 2 Ave|     379.0|
| Bank St & Hudson St|     643.0|
|Park Pl & Vanderb...|     340.0|
|Meserole Ave & Ma...|     416.0|
|       44 Dr & 21 St|     245.0|
|    Jay St & Tech Pl|     504.0|
|Central Park West...|     915.0|
|Sterling St & Bed...|      46.0|
|Carlton Ave & Flu...|     255.0|
|  E 91 St & Park Ave|     221.0|
|39 St & 2 Ave - C...|      89.0|
|  Broadway & W 56 St|    1595.0|
+--------------------+----------+
only showing top 20 rows



Среднее количество завершения поездок в дневной период для каждой станции (12PM - 6PM)

In [75]:
from pyspark.sql.functions import date_format
from pyspark.sql import functions as F
end_day_spark_df = spark_df.withColumn("stoptimet", date_format("stoptime", 'HH:mm:ss'))\
.filter(F.col("stoptimet") >= "12:00:00")\
.filter(F.col("stoptimet") <= "17:59:00")\
.groupBy("end station name")\
.count()\
.groupBy("end station name")\
.avg("count")

end_day_spark_df.show()

+--------------------+----------+
|    end station name|avg(count)|
+--------------------+----------+
|       10 St & 5 Ave|     147.0|
|  N 11 St & Kent Ave|     149.0|
|       45 Rd & 11 St|     207.0|
|    W 12 St & W 4 St|     832.0|
|   Dean St & Hoyt St|     227.0|
| Berkeley Pl & 6 Ave|     203.0|
| Park Ave & E 124 St|     519.0|
|      49 Ave & 21 St|     119.0|
|     E 91 St & 2 Ave|     474.0|
|Sterling St & Bed...|      61.0|
|Meserole Ave & Ma...|     442.0|
|    Jay St & Tech Pl|     414.0|
| Bank St & Hudson St|     884.0|
|Central Park West...|    1147.0|
|Park Pl & Vanderb...|     307.0|
|       44 Dr & 21 St|     225.0|
|Carlton Ave & Flu...|     224.0|
|  E 91 St & Park Ave|     268.0|
|39 St & 2 Ave - C...|      77.0|
|  Broadway & W 56 St|    1430.0|
+--------------------+----------+
only showing top 20 rows



Среднее количество начала поездок в вечерний период для каждой станции (6PM - 12AM)

In [76]:
from pyspark.sql import functions as F
start_evening_spark_df = spark_df.withColumn("starttimet", date_format("starttime", 'HH:mm:ss'))\
.filter(F.col("starttimet") >= "18:00:00")\
.filter(F.col("starttimet") <= "23:59:00")\
.groupBy("start station name")\
.count()\
.groupBy("start station name")\
.avg("count")

start_evening_spark_df.show()

+--------------------+----------+
|  start station name|avg(count)|
+--------------------+----------+
|  N 11 St & Kent Ave|     135.0|
|       45 Rd & 11 St|     238.0|
|       10 St & 5 Ave|      75.0|
| Park Ave & E 124 St|     442.0|
|    W 12 St & W 4 St|     403.0|
|      49 Ave & 21 St|      29.0|
|   Dean St & Hoyt St|     113.0|
| Berkeley Pl & 6 Ave|     102.0|
|     E 91 St & 2 Ave|     224.0|
|    Jay St & Tech Pl|     290.0|
|Central Park West...|     322.0|
| Bank St & Hudson St|     336.0|
|Park Pl & Vanderb...|     269.0|
|Meserole Ave & Ma...|     301.0|
|       44 Dr & 21 St|     259.0|
|Sterling St & Bed...|      18.0|
|Carlton Ave & Flu...|     120.0|
|  E 91 St & Park Ave|     196.0|
|39 St & 2 Ave - C...|      43.0|
|Washington St & G...|     516.0|
+--------------------+----------+
only showing top 20 rows



Среднее количество завершения поездок в вечерний период для каждой станции (6PM - 12AM)

In [77]:
from pyspark.sql.functions import date_format
from pyspark.sql import functions as F
end_evening_spark_df = spark_df.withColumn("stoptimet", date_format("stoptime", 'HH:mm:ss'))\
.filter(F.col("stoptimet") >= "18:00:00")\
.filter(F.col("stoptimet") <= "23:59:00")\
.groupBy("end station name")\
.count()\
.groupBy("end station name")\
.avg("count")

end_evening_spark_df.show()

+--------------------+----------+
|    end station name|avg(count)|
+--------------------+----------+
|       10 St & 5 Ave|      85.0|
|       45 Rd & 11 St|     141.0|
|  N 11 St & Kent Ave|     121.0|
| Park Ave & E 124 St|     233.0|
|   Dean St & Hoyt St|     115.0|
| Berkeley Pl & 6 Ave|     152.0|
|    W 12 St & W 4 St|     497.0|
|      49 Ave & 21 St|      52.0|
|     E 91 St & 2 Ave|     392.0|
|    Jay St & Tech Pl|     222.0|
|Central Park West...|     432.0|
|Meserole Ave & Ma...|     326.0|
|Park Pl & Vanderb...|     216.0|
| Bank St & Hudson St|     457.0|
|       44 Dr & 21 St|     132.0|
|Sterling St & Bed...|      43.0|
|  E 91 St & Park Ave|     178.0|
|Carlton Ave & Flu...|     132.0|
|39 St & 2 Ave - C...|      41.0|
|Carlton Ave & Dea...|     247.0|
+--------------------+----------+
only showing top 20 rows



Среднее количество начала поездок в ночной период для каждой станции (12AM - 6AM)

In [78]:
from pyspark.sql import functions as F
start_night_spark_df = spark_df.withColumn("starttimet", date_format("starttime", 'HH:mm:ss'))\
.filter(F.col("starttimet") >= "00:00:00")\
.filter(F.col("starttimet") <= "05:59:00")\
.groupBy("start station name")\
.count()\
.groupBy("start station name")\
.avg("count")

start_night_spark_df.show()

+--------------------+----------+
|  start station name|avg(count)|
+--------------------+----------+
|       45 Rd & 11 St|       6.0|
|  N 11 St & Kent Ave|      41.0|
|       10 St & 5 Ave|       3.0|
| Berkeley Pl & 6 Ave|      17.0|
|    W 12 St & W 4 St|      33.0|
|   Dean St & Hoyt St|      14.0|
|      49 Ave & 21 St|       4.0|
| Park Ave & E 124 St|      17.0|
|     E 91 St & 2 Ave|      19.0|
|       44 Dr & 21 St|      13.0|
|Central Park West...|      34.0|
| Bank St & Hudson St|      48.0|
|Meserole Ave & Ma...|      30.0|
|    Jay St & Tech Pl|      13.0|
|Park Pl & Vanderb...|      44.0|
|Carlton Ave & Flu...|      18.0|
|  E 91 St & Park Ave|      12.0|
|39 St & 2 Ave - C...|       1.0|
|  Broadway & W 56 St|      45.0|
|Washington St & G...|      46.0|
+--------------------+----------+
only showing top 20 rows



Среднее количество завершения поездок в ночной период для каждой станции (12AM - 6AM)

In [79]:
from pyspark.sql.functions import date_format
from pyspark.sql import functions as F
end_night_spark_df = spark_df.withColumn("stoptimet", date_format("stoptime", 'HH:mm:ss'))\
.filter(F.col("stoptimet") >= "00:00:00")\
.filter(F.col("stoptimet") <= "05:59:00")\
.groupBy("end station name")\
.count()\
.groupBy("end station name")\
.avg("count")

end_night_spark_df.show()

+--------------------+----------+
|    end station name|avg(count)|
+--------------------+----------+
|  N 11 St & Kent Ave|       7.0|
|       45 Rd & 11 St|       6.0|
|       10 St & 5 Ave|       7.0|
| Park Ave & E 124 St|      43.0|
|   Dean St & Hoyt St|       5.0|
|    W 12 St & W 4 St|      27.0|
|      49 Ave & 21 St|       1.0|
|     E 91 St & 2 Ave|      53.0|
|Meserole Ave & Ma...|      54.0|
|Central Park West...|      45.0|
| Bank St & Hudson St|      46.0|
|    Jay St & Tech Pl|       6.0|
|Park Pl & Vanderb...|      18.0|
|       44 Dr & 21 St|       3.0|
|Sterling St & Bed...|       6.0|
|Carlton Ave & Flu...|      11.0|
|  E 91 St & Park Ave|      13.0|
|39 St & 2 Ave - C...|       2.0|
|   E 2 St & Avenue C|      89.0|
|Washington St & G...|      19.0|
+--------------------+----------+
only showing top 20 rows



In [86]:
data_start_day = start_day_spark_df.select("avg(count)").collect()
data_start_day = [l[0] for l in data_start_day]
data_start_morning = start_morning_spark_df.select("avg(count)").collect()
data_start_morning = [l[0] for l in data_start_morning]
data_start_evening = start_evening_spark_df.select("avg(count)").collect()
data_start_evening = [l[0] for l in data_start_evening]
data_start_night = start_night_spark_df.select("avg(count)").collect()
data_start_night = [l[0] for l in data_start_night]
data_end_day = end_day_spark_df.select("avg(count)").collect()
data_end_day = [l[0] for l in data_end_day]
data_end_morning = end_morning_spark_df.select("avg(count)").collect()
data_end_morning = [l[0] for l in data_end_morning]
data_end_evening = end_evening_spark_df.select("avg(count)").collect()
data_end_evening = [l[0] for l in data_end_evening]
data_end_night = end_night_spark_df.select("avg(count)").collect()
data_end_night = [l[0] for l in data_end_night]


In [87]:
from datetime import datetime
time_index = [
    datetime.strptime("06:00:00", '%H:%M:%S').time() for k in range(len(data_start_morning) + len(data_end_morning))
] + \
[
    datetime.strptime("12:00:00", '%H:%M:%S').time() for k in range(len(data_start_day) + len(data_end_day))
] + \
[
    datetime.strptime("18:00:00", '%H:%M:%S').time() for k in range(len(data_start_evening) + len(data_end_evening))
] + \
[
    datetime.strptime("00:00:00", '%H:%M:%S').time() for k in range(len(data_start_night) + len(data_end_night))
]
data = data_start_morning + data_end_morning + data_start_day + data_end_day + data_start_evening + data_end_evening + data_start_night + data_end_night

In [88]:
import folium
import folium.plugins as plugins
m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)

hm = plugins.HeatMapWithTime(
    data,
    index=time_index,
    auto_play=True,
    max_opacity=0.3
)

hm.add_to(m)

m